In [557]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

In [558]:
def concatenate_merged_dfs(co2_dir, pivot_df, year_range):
    """
    Concatenate all merged DataFrames into one DataFrame.
    
    Parameters:
        co2_dir (str): Directory containing the CO2 emissions data files.
        pivot_df (pd.DataFrame): Pivot table of disaster events data.
        year_range (tuple): Range of years to include in the plot.
    
    Returns:
        pd.DataFrame: Concatenated DataFrame of merged data.
    """
    merged_dfs = []

    for co2_file in os.listdir(co2_dir):
        if co2_file.endswith('.csv'):
            if 'District of Columbia' in co2_file:
                continue
            # Extract the state name from the CO2 emission file name
            state_name = os.path.basename(co2_file).split('-')[0]
            file_path = os.path.join(co2_dir, co2_file)
            # Load the CO2 emissions data
            co2_df = pd.read_csv(file_path)
            
            # Ensure the 'year' column is of the correct data type
            co2_df['datetime'] = pd.to_datetime(co2_df['datetime'])
            co2_df['year'] = co2_df['datetime'].dt.year.astype(int)

            # Group by 'year' and sum the 'sum' column
            co2_df = co2_df.groupby('year')['sum'].sum().reset_index(name='total_co2_emissions')

            pivot_df.index = pivot_df.index.astype(int)
            # Filter the pivot table to include only the years 2010 to 2021
            pivot_df_filtered = pivot_df[(pivot_df.index >= year_range[0]) & (pivot_df.index <= year_range[1])]
            pivot_df_filtered = pivot_df_filtered[state_name]
            # rename the one column to disaster_count
            pivot_df_filtered = pivot_df_filtered.rename('disaster_count')

            # Merge the disaster events data with the CO2 emissions data
            merged_df = pd.merge(co2_df, pivot_df_filtered, left_on='year', right_index=True, how='inner')
            merged_df['state'] = state_name
            print(merged_df)
            merged_dfs.append(merged_df)

    # Vertically concatenate all the merged DataFrames
    concatenated_df = pd.concat(merged_dfs)
    return concatenated_df

In [559]:
pivot_df = pd.read_csv(r'D:\OneDrive\StudyMat\CarbonSci-cle-Data\data\processed\disaster\disaster_counts.csv', index_col=0)

In [560]:
df = concatenate_merged_dfs(r'D:\OneDrive\StudyMat\CarbonSci-cle-Data\data\raw\co2\total', pivot_df, (2010, 2021))

    year  total_co2_emissions  disaster_count    state
0   2010           42521084.0               3  Alabama
1   2011           42824908.0              67  Alabama
2   2012           39787240.0              11  Alabama
3   2013           37981200.0               0  Alabama
4   2014           39175640.0               0  Alabama
5   2015           38039592.0               0  Alabama
6   2016           36381056.0               0  Alabama
7   2017           35198304.0             108  Alabama
8   2018           35076212.0              18  Alabama
9   2019           32909928.0               0  Alabama
10  2020           29739268.0               0  Alabama
11  2021           32203132.0               0  Alabama
    year  total_co2_emissions  disaster_count    state
0   2010           25086588.0               9  Arizona
1   2011           24671428.0               4  Arizona
2   2012           24444662.0               0  Arizona
3   2013           25619100.0               3  Arizona
4   2014  

In [561]:
# Emission growth rate for each state
df['emission_growth_rate'] = df.groupby('state')['total_co2_emissions'].pct_change() * 100

In [562]:
# Lag the disaster count by one year
df['disaster_count_lagged'] = df.groupby('state')['disaster_count'].shift(1)
df.fillna(0, inplace=True)

# Lag the emission growth rate by one year
df['emission_growth_rate_lagged'] = df.groupby('state')['emission_growth_rate'].shift(1)

# Lag the co2 emissions by one year
df['total_co2_emissions_lagged'] = df.groupby('state')['total_co2_emissions'].shift(1)

In [563]:
df['years_since'] = df['year'] - 2010  # Years since 2010

In [564]:
# Moving average of the disaster count with a window of 3 years
df['disaster_count_moving_avg'] = df.groupby('state')['disaster_count'].transform(lambda x: x.rolling(3, min_periods=1).mean())

# Round the disaster count moving average to nearest integer
df['disaster_count_moving_avg'] = df['disaster_count_moving_avg'].round().astype(int)

# Moving average of the CO2 emissions with a window of 3 years
df['total_co2_emissions_moving_avg'] = df.groupby('state')['total_co2_emissions'].transform(lambda x: x.rolling(3, min_periods=1).mean())

# Round the CO2 emissions moving average to nearest integer
df['total_co2_emissions_moving_avg'] = df['total_co2_emissions_moving_avg'].round().astype(int)

In [565]:
from sklearn.preprocessing import LabelEncoder

# Encode State as a number
le = LabelEncoder()
df['State_Encoded'] = le.fit_transform(df['state'])
df.drop(columns=['state'], inplace=True)


In [582]:
# Save the state encoding mapping to a CSV file
state_encoding = pd.DataFrame({'state': le.classes_, 'State_Encoded': le.transform(le.classes_)})
state_encoding.to_csv(r'D:\OneDrive\StudyMat\CarbonSci-cle-Data\state_encoding.csv', index=False)

In [566]:
# Create a binary target
df['Disaster_Binary'] = df['disaster_count_moving_avg'].apply(lambda x: 1 if x > 0 else 0)

In [567]:
# Reset the index
df.reset_index(drop=True, inplace=True)

# Drop na
df.dropna(inplace=True)

In [568]:
train_data = df[df['year'] < 2020]
test_data = df[df['year'] >= 2020]

# Reset index
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [569]:
from sklearn.model_selection import train_test_split

# Define feature columns (excluding Year and target 'Disasters')
features = ['total_co2_emissions_lagged', 'years_since', 'State_Encoded']

# Split train and test data for regression
X_train = train_data[features]
y_train = train_data['disaster_count']


# Split train and test data for binary classification
y_train_binary = train_data['Disaster_Binary']
y_test_binary = test_data['Disaster_Binary']  
X_train_binary = train_data[features]


# Split train and test data for XGBoost
X_train_xgboost = train_data[train_data['Disaster_Binary'] == 1][features]
y_train_xgboost = train_data[train_data['Disaster_Binary'] == 1]['disaster_count']
X_test = test_data[features]
y_test = test_data['disaster_count']

In [570]:
from sklearn.ensemble import RandomForestClassifier

# Train a classifier
classifier = RandomForestClassifier()
classifier.fit(X_train_binary, y_train_binary)

# Predict probabilities of being non-zero
non_zero_probs = classifier.predict_proba(X_test)[:, 1]

# Predict binary target
y_pred_binary = classifier.predict(X_test)

from sklearn.metrics import classification_report   

# Print classification report
print(classification_report(y_test_binary, y_pred_binary))


              precision    recall  f1-score   support

           0       0.93      0.80      0.86        69
           1       0.62      0.85      0.72        27

    accuracy                           0.81        96
   macro avg       0.78      0.82      0.79        96
weighted avg       0.84      0.81      0.82        96



In [586]:
import xgboost as xgb

# Train regression model on non-zero data only
reg_model = xgb.XGBRegressor(objective='count:poisson', n_estimators=700)
reg_model.fit(X_train_xgboost, y_train_xgboost)

# Combine the two models for final prediction
final_predictions = []
for i, prob in enumerate(non_zero_probs):
    if prob > 0.5:
        # Predict the disaster count using regression for non-zero cases
        final_predictions.append((reg_model.predict(X_test[i:i+1])[0]))
    else:
        # Predict zero for cases with low non-zero probability
        final_predictions.append(0)

final_predictions = np.round(final_predictions).astype(int)

In [587]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(y_test, final_predictions)

print(f"MAE: {mae}")



MAE: 9.552083333333334


In [588]:
# Define the acceptable error margin (e.g., 10%)
error_margin = 0.1  # 10%

# Calculate the accuracy
accurate_predictions = np.abs(y_test - final_predictions) <= (error_margin * y_test)
accuracy_percentage = np.mean(accurate_predictions) * 100

print(f"Accuracy Percentage: {accuracy_percentage:.2f}%")

Accuracy Percentage: 75.00%


In [573]:
import xgboost as xgb


# Train XGBoost model
model = xgb.XGBRegressor(objective='count:poisson', n_estimators=700, learning_rate=0.1)

model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Round the predictions to the nearest integer
predictions = np.round(predictions)

In [574]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(y_test, predictions)

print(f"MAE: {mae}")


MAE: 9.427083333333334


In [576]:
# Define the acceptable error margin (e.g., 10%)
error_margin = 0.1  # 10%

# Calculate the accuracy
accurate_predictions = np.abs(y_test - predictions) <= (error_margin * y_test)
accuracy_percentage = np.mean(accurate_predictions) * 100

print(f"Accuracy Percentage: {accuracy_percentage:.2f}%")

Accuracy Percentage: 71.88%


In [579]:
import numpy as np
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import joblib

class DisasterPredictionPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.5, n_estimators=700):
        self.threshold = threshold
        self.n_estimators = n_estimators
        self.classifier = RandomForestClassifier()
        self.reg_model = xgb.XGBRegressor(objective='count:poisson', n_estimators=self.n_estimators)
        self.non_zero_probs = None

    def fit(self, X_train_binary, y_train_binary, X_train_xgboost, y_train_xgboost):
        # Train the classifier
        self.classifier.fit(X_train_binary, y_train_binary)
        
        # Predict probabilities of being non-zero
        self.non_zero_probs = self.classifier.predict_proba(X_train_xgboost)[:, 1]
        
        # Train the regression model on non-zero data only
        self.reg_model.fit(X_train_xgboost, y_train_xgboost)
        return self

    def predict(self, X_test):
        # Predict probabilities of being non-zero
        non_zero_probs = self.classifier.predict_proba(X_test)[:, 1]
        
        final_predictions = []
        for i, prob in enumerate(non_zero_probs):
            if prob > self.threshold:
                # Predict the disaster count using regression for non-zero cases
                final_predictions.append(self.reg_model.predict(X_test[i:i+1])[0])
            else:
                # Predict zero for cases with low non-zero probability
                final_predictions.append(0)
        return np.round(final_predictions).astype(int)

# Example usage
# Assuming X_train_binary, y_train_binary, X_train_xgboost, y_train_xgboost, X_test_binary, and X_test_xgboost are already defined

# Create and train the pipeline
pipeline = DisasterPredictionPipeline(threshold=0.5, n_estimators=700)
pipeline.fit(X_train_binary, y_train_binary, X_train_xgboost, y_train_xgboost)

# Make predictions
final_predictions = pipeline.predict(X_test)

# Save the trained pipeline
joblib.dump(pipeline, 'disaster_prediction_pipeline.pkl')

# Load the trained pipeline (for use in the backend)
loaded_pipeline = joblib.load('disaster_prediction_pipeline.pkl')
backend_predictions = loaded_pipeline.predict(X_test)